# Backtest with Prompt Optimization (2010-2023)

This notebook demonstrates Phase 1 of the workflow:
1.  **Backtest**: Running the agentic pipeline over historical data (2010-2023).
2.  **Training/Optimization**: Iteratively refining the instructions (prompts) for each sub-agent (Alpha, Risk, Portfolio) based on backtest performance.
3.  **Saving**: Persisting the optimized prompts for use in the out-of-sample test.

### Prerequisites
Ensure you have the environment variables set for `OPENAI_API_KEY`, `ALPACA_API_KEY`, and `ALPACA_SECRET_KEY`.

In [1]:
import os
import sys
import json
import pandas as pd
from datetime import datetime
from pathlib import Path

# Add project root to path to import Orchestrator
project_root = Path("../").resolve()
sys.path.insert(0, str(project_root))
sys.path.insert(0, str(project_root / "FinAgents" / "orchestrator_demo"))
sys.path.insert(0, str(project_root / "FinAgents" / "agent_pools"))

# Import Orchestrator
from FinAgents.orchestrator_demo.orchestrator import Orchestrator

# Initialize Orchestrator
orchestrator = Orchestrator()
print("✅ Orchestrator Initialized")

Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
Users of this version of Gym should be able to simply replace 'import gym' with 'import gymnasium as gym' in the vast majority of cases.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.
[59256:MainThread](2025-11-30 02:03:10,835) INFO - qlib.Initialization - [config.py:452] - default_conf: client.
[59256:MainThread](2025-11-30 02:03:10,836) WARNING - qlib.Initialization - [__init__.py:65] - auto_path is False, please make sure None is mounted
[59256:MainThread](2025-11-30 02:03:10,838) INFO - qlib.Initialization - [__init__.py:75] - qlib successfully initialized based on client settings.
[59256:MainThread](2025-11-30 02:03:10,839) INFO - qlib.Initialization - [__init__.py:77] -

✅ Qlib system initialized successfully
✅ Qlib components loaded successfully
✅ Orchestrator Initialized


### Step 1: Define Optimization Loop

We simulate a training loop where we run a backtest for a specific year, evaluate performance, and ask a "Meta-Agent" to improve the instructions if targets aren't met.

In [ ]:
def train_agents_over_period(symbol, start_year, end_year):
    current_prompts = {
        "Alpha": orchestrator.alpha_agent.agent.instructions,
        "Risk": orchestrator.risk_agent.agent.instructions,
        "Portfolio": orchestrator.portfolio_agent.agent.instructions
    }
    
    performance_history = []
    
    for year in range(start_year, end_year + 1):
        start_date = f"{year}-01-01"
        end_date = f"{year}-12-31"
        print(f"\n--- Processing Year: {year} ---")
        
        # Run Pipeline (using the Legacy pipeline method for direct control, or agentic if preferred)
        # Here we use the underlying run_pipeline logic exposed in Orchestrator
        # Note: In a real scenario, we would capture the result object
        try:
            result = orchestrator.run_pipeline(symbol, start_date, end_date, mode="backtest")
            
            if result and result.get('status') == 'success':
                metrics = result.get('performance_metrics', {})
                sharpe = metrics.get('sharpe_ratio', 0.0)
                print(f"📊 Performance for {year}: Sharpe Ratio = {sharpe:.2f}")
                
                performance_history.append({'year': year, 'sharpe': sharpe})
                
                # Optimization Logic: If performance is poor, optimize prompts
                if sharpe < 1.0: # Threshold for optimization
                    print("⚠️ Performance below threshold. Optimizing prompts...")
                    
                    # Call the optimizer (Meta-Agent)
                    # In the demo, this calls OpenAI to rewrite instructions
                    new_instruction = orchestrator.optimize_agent_prompts(
                        agent_name="Alpha", 
                        performance_metric="Sharpe Ratio", 
                        current_value=sharpe, 
                        target_value=1.5
                    )
                    
                    if new_instruction and "Optimization failed" not in new_instruction:
                         current_prompts["Alpha"] = new_instruction
                         print("✅ Alpha Agent prompt updated.")
            else:
                print(f"❌ Backtest failed for {year}: {result.get('message') if result else 'Unknown error'}")
                
        except Exception as e:
            print(f"❌ Error during execution: {e}")
            
    return current_prompts, performance_history

# Run the Training Loop
symbol = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'NVDA', 'META', 'TSLA', 'JPM', 'V', 'WMT']
optimized_prompts, history = train_agents_over_period(symbol, 2019, 2023)

2025-11-30 02:03:12,330 - Orchestrator - INFO - Running pipeline for AAPL from 2010-01-01 to 2010-12-31
2025-11-30 02:03:12,331 - Orchestrator - INFO - Fetching data for ['AAPL'] from 2010-01-01 00:00:00 to 2010-12-31 00:00:00



--- Processing Year: 2010 ---
DEBUG: 🤖 Requesting Alpha Agent LLM...


2025-11-30 02:03:15,017 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"


DEBUG: 🛠️ run_alpha_pipeline TOOL INVOKED by LLM


2025-11-30 02:03:17,377 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"


...

DEBUG: Risk LLM finished. Context keys: ['data', 'market_returns', 'risk_metrics', 'data_processor', 'result']
🚀 Running simple backtest with paper interface design
DEBUG: Predictions received. Shape: (242,)
DEBUG: Predictions sample:
datetime    instrument
2020-01-28  AAPL         -1.0
2020-01-29  AAPL         -1.0
2020-01-30  AAPL         -1.0
2020-01-31  AAPL         -1.0
2020-02-03  AAPL         -1.0
dtype: float64
DEBUG: Predictions stats:
count    242.000000
mean      -0.099174
std        0.997132
min       -1.000000
25%       -1.000000
50%       -1.000000
75%        1.000000
max        1.000000
dtype: float64
   Period: 2020-01-01 to 2020-12-31
   Look-back: 20 days, Horizon: 5 days
DEBUG: Market returns lookup prepared. Size: 262
   📊 Returns series length: 242
   📊 Costs series length: 242
   📊 Returns stats: mean=-0.000255, std=0.010336, min=-0.022126, max=0.028303
   💰 Costs stats: mean=0.000008, total=0.002000, min=0.000000, max=0.002000
   📈 Cost-adjusted: mean=-0.000263, 

KeyboardInterrupt: 

2025-11-30 02:04:56,112 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/traces/ingest "HTTP/1.1 204 No Content"
2025-11-30 02:04:56,290 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"


DEBUG: 🛠️ run_alpha_pipeline TOOL INVOKED by LLM


2025-11-30 02:04:58,601 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
2025-11-30 02:05:01,680 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/traces/ingest "HTTP/1.1 204 No Content"


### Step 2: Save Optimized Prompts

Save the evolved instructions to a file so they can be loaded for the out-of-sample test.

In [3]:
output_path = "optimized_prompts.json"
with open(output_path, "w") as f:
    json.dump(optimized_prompts, f, indent=2)
    
print(f"💾 Optimized prompts saved to {output_path}")
print("History:", history)

💾 Optimized prompts saved to optimized_prompts.json
History: []
